In [2]:
import pandas as pd
import asyncio, json
from asyncio import Semaphore
from aiohttp import ClientSession, TCPConnector

df = pd.read_excel("prices3.xlsx")
df.columns = [c.lower().strip() for c in df.columns]
df["sku"] = df["sku"].astype(str)

Sku_ProductId_Api  = "https://ksa-api.boutiqaat.com/searchplus/rest/V2/global/suggest"
ProductId_Price_Api  = "https://ksa-api.boutiqaat.com/searchplus/rest/V2/productinfo?useMultiConfigResponse=true"

HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "origin": "https://www.boutiqaat.com",
    "referer": "https://www.boutiqaat.com/",
    "user-agent": "Mozilla/5.0",
    "param": '{"gender":"4194","app_version":"8.0.0","platform":"web","device_type":"desktop"}'
}

BASE_PAYLOAD = {
    "countryCodeAndLanguage": "kw_en",
    "numberOfRecords": 20,
    "featuredOnly": None,
    "searchString": ""
}

BASE_PAYLOAD_2 = {
    "productId": "",
    "ruleId": "",
    "celebrityId": "0",
    "optionId": "",
    "countryCodeAndLanguage": "kw_en"
}

# limit concurrency to 50 requests at a time, controlling how many request access the shared resources by how many threads at the same time
SEM = Semaphore(50)

async def safe_post(session, url, headers, payload):
    async with SEM:
        async with session.post(url, headers=headers, json=payload) as r:
            return await r.json(content_type=None)

async def safe_get(session, url):
    async with SEM:
        async with session.get(url, ssl=False) as r:
            return await r.text()

async def fectch_productId(session, sku):
    payload = BASE_PAYLOAD.copy()
    payload["searchString"] = sku
    j = await safe_post(session, Sku_ProductId_Api, HEADERS, payload)
    try:
        return j["data"][0]["data"][0]["productId"]
    except:
        return None

async def fetch_price(session, productId):
    payload = BASE_PAYLOAD_2.copy()
    payload["productId"] = productId
    j = await safe_post(session, ProductId_Price_Api, HEADERS, payload)
    try:
        return float(j[0]["final_price_with_tax"])
    except:
        return None

async def refresh_price(session, productId):
    url = f"https://ksa-api.boutiqaat.com/cdc/rest/V1/realtime/{productId}"
    text = await safe_get(session, url)
    return "success" if "success" in text.lower() else "failed"

async def process_row(session, sku, expected_price):
    productId = await fectch_productId(session, sku)
    if not productId:
        return {
            "sku": sku,
            "excel_price": expected_price,
            "web_price": None,
            "match": None,
            "refresh_status": "failed"
        }

    web_price = await fetch_price(session, productId)
    match = (web_price == expected_price) if web_price is not None else None
    refresh_status = "-"

    if match is False:
        refresh_status = await refresh_price(session, productId)
        if refresh_status == "success":
            await asyncio.sleep(0.1)  # small delay for cache to update, i might delete this idk
            web_price = await fetch_price(session, productId)
            match = (web_price == expected_price) if web_price is not None else None

    return {
        "sku": sku,
        "excel_price": expected_price,
        "web_price": web_price,
        "match": match,
        "refresh_status": refresh_status
    }

async def main(df):
    connector = TCPConnector(limit=100)  # --> 100 concurrent open tcp sockets WHY ?? 
    async with ClientSession(connector=connector) as s:
        return await asyncio.gather(*(
            process_row(s, sku, float(price))
            for sku, price in zip(df["sku"], df["special price"])
        ))

out = await main(df)
results = pd.DataFrame(out)

# Save
results.to_excel("comparison_results.xlsx", index=False)
results.to_csv("comparison_results.csv", index=False)

results.head(20)


,sku,excel_price,web_price,match,refresh_status
0,ORL-00006341,6.600,6.60,True,-
1,ORL-00006341,6.601,6.60,False,success
2,I-00000190608,15.000,15.00,True,-
3,I-00000190608,15.001,15.00,False,success
4,ORL-00006239,22.500,18.00,False,success
5,ORL-00006239,22.501,18.00,False,success
6,ORL-00004724,17.130,17.13,True,-
7,ORL-00004724,17.131,17.13,False,success
8,MU-00022119,8.500,8.50,True,-
9,MU-00022119,8.501,8.50,False,success
